In [1]:
import numpy as np
import re
from os.path import join, relpath
import pandas as pd
from openpyxl import load_workbook
import math

#csv_file = df = pd.read_csv('C:/Users/Matsumoto/OneDrive - Kyoto University/Project/MSAD/msad_crconi.csv',
                            #encoding = 'UTF-8')
#output_path = 'C:/Users/Matsumoto/OneDrive - Kyoto University/Project/MSAD/msad_crconi.csv'
split_a = -1
split_b = -4
file = r'/home/wang/MSAD/OUTPUTS/MSAD_1.xlsx'

def read(incar_dir):
    with open(incar_dir) as f:
        input_strip = [s.strip() for s in f.readlines()]
    return input_strip

def get_coor(input_dir, tline = 8):
    with open(input_dir) as f:
        input_strip = [s.strip() for s in f.readlines()]
    
    def trans2list(i):
        s = ''.join(input_strip[i])
        strip = s.split()
        strip = [float(x) for x in strip[:3]]
        return strip

    s = ''.join(input_strip[6])
    strip_6 = s.split()
    strip_6 = [float(x) for x in strip_6]
    li_sum = int(sum(strip_6))
    split_pattern = '[/\\\]'
    type_dir = re.split(split_pattern, input_dir)[-1]
    print(f'Num of atoms in {type_dir}: {li_sum}')

    input_coor = []

    for i in range(tline, tline + li_sum):
        strip = trans2list(i)
        input_coor += strip

    input_coor = np.array(input_coor).reshape(li_sum, 3)

    assert len(input_coor) == li_sum

    return li_sum, strip_6, input_coor

def msad_ana(cont_dir, pos_dir, osz_dir, lattice_param, cell_num):
    param = {}
    lattice_param = lattice_param * cell_num

    li_sum_cont, strip_cont, cont_coor = get_coor(cont_dir)
    li_sum_pos, strip_pos, pos_coor = get_coor(pos_dir)
    print(f'Num of Cr, Co, Ni: {strip_pos}')
    param['Total_num'] = li_sum_pos
    param['CrCoNi'] = strip_pos

    cont_coor = cont_coor * lattice_param #Lattice parameter * cell units
    pos_coor = pos_coor * lattice_param
    divide = 10000 / li_sum_pos
    cr_num, co_num, ni_num = int(strip_pos[0]), int(strip_pos[1]), int(strip_pos[2])
    divide_cr, divide_co, divide_ni = 10000 / cr_num, 10000 / co_num, 10000 / ni_num, 

    msad = np.sum(np.power((cont_coor - pos_coor), 2)) * divide
    msad_cr = np.sum(np.power(cont_coor[:cr_num] - pos_coor[:cr_num], 2)) * divide_cr
    msad_co = np.sum(np.power(cont_coor[cr_num:cr_num+co_num] - pos_coor[cr_num:cr_num+co_num], 2)) * divide_co
    msad_ni = np.sum(np.power(cont_coor[cr_num+co_num:] - pos_coor[cr_num+co_num:], 2)) * divide_ni

    param['MSAD'] = round(msad,4)
    param['cr_msad'] = round(msad_cr,4)
    param['co_msad'] = round(msad_co,4)
    param['ni_msad'] = round(msad_ni,4)
    
    print('MSAD value: ' + str(round(msad,4)) + ' pm^2')
    print(f'MSAD for Cr: {round(msad_cr,4)} pm^2\nMSAD for Co: {round(msad_co,4)} pm^2\nMSAD for Ni: {round(msad_ni,4)} pm^2\n')
    
    oszicar = read(osz_dir)[split_a]
    e_ground = ''.join(oszicar).split()[split_b]
    print(e_ground)
    
    return param

def msad_ana_bi(cont_dir, pos_dir, osz_dir, lattice_param, cell_num):
    param = {}
    lattice_param = lattice_param * cell_num

    li_sum_cont, strip_cont, cont_coor = get_coor(cont_dir)
    li_sum_pos, strip_pos, pos_coor = get_coor(pos_dir)
    print(f'Num of A, B: {strip_pos}')
    param['Total_num'] = li_sum_pos
    param['AB'] = strip_pos

    cont_coor = cont_coor * lattice_param #Lattice parameter * cell units
    pos_coor = pos_coor * lattice_param
    divide = 10000 / li_sum_pos
    cr_num, co_num = int(strip_pos[0]), int(strip_pos[1])
    divide_cr, divide_co = 10000 / cr_num, 10000 / co_num

    msad = np.sum(np.power((cont_coor - pos_coor), 2)) * divide
    msad_cr = np.sum(np.power(cont_coor[:cr_num] - pos_coor[:cr_num], 2)) * divide_cr
    msad_co = np.sum(np.power(cont_coor[cr_num:cr_num+co_num] - pos_coor[cr_num:cr_num+co_num], 2)) * divide_co

    param['MSAD'] = round(msad,4)
    param['cr_msad'] = round(msad_cr,4)
    param['co_msad'] = round(msad_co,4)
    
    print('MSAD value: ' + str(round(msad,4)) + ' pm^2')
    print(f'MSAD for A: {round(msad_cr,4)} pm^2\nMSAD for B: {round(msad_co,4)}')
    
    oszicar = read(osz_dir)[split_a]
    e_ground = ''.join(oszicar).split()[split_b]
    print(e_ground)
    
    return param

def msad_ana_qua(cont_dir, pos_dir, osz_dir, lattice_param, cell_num):
    param = {}
    lattice_param = lattice_param * cell_num

    li_sum_cont, strip_cont, cont_coor = get_coor(cont_dir)
    li_sum_pos, strip_pos, pos_coor = get_coor(pos_dir)
    print(f'Num of Cr, Mn, Co, Ni: {strip_pos}')
    param['Total_num'] = li_sum_pos
    param['CrMnCoNi'] = strip_pos

    cont_coor = cont_coor * lattice_param #Lattice parameter * cell units
    pos_coor = pos_coor * lattice_param
    divide = 10000 / li_sum_pos
    cr_num, mn_num, co_num, ni_num = int(strip_pos[0]), int(strip_pos[1]), int(strip_pos[2]), int(strip_pos[3])
    divide_cr, divide_mn, divide_co, divide_ni = 10000 / cr_num, 10000 / mn_num, 10000 / co_num, 10000 / ni_num, 

    msad = np.sum(np.power((cont_coor - pos_coor), 2)) * divide
    msad_cr = np.sum(np.power(cont_coor[:cr_num] - pos_coor[:cr_num], 2)) * divide_cr
    msad_mn = np.sum(np.power(cont_coor[cr_num:cr_num+mn_num] - pos_coor[cr_num:cr_num+mn_num], 2)) *divide_mn
    msad_co = np.sum(np.power(cont_coor[cr_num+mn_num:cr_num+mn_num+co_num] 
                              - pos_coor[cr_num+mn_num:cr_num+mn_num+co_num], 2)) * divide_co
    msad_ni = np.sum(np.power(cont_coor[cr_num+mn_num+co_num:] - pos_coor[cr_num+mn_num+co_num:], 2)) * divide_ni

    param['MSAD'] = round(msad,4)
    param['cr_msad'] = round(msad_cr, 4)
    param['mn_msad'] = round(msad_mn, 4)
    param['co_msad'] = round(msad_co, 4)
    param['ni_msad'] = round(msad_ni, 4)
    
    print('MSAD value: ' + str(round(msad, 4)) + ' pm^2')
    print(f'MSAD for Cr: {round(msad_cr, 4)} pm^2\nMSAD for Mn: {round(msad_mn,4)} pm^2\nMSAD for Co: {round(msad_co,4)} pm^2\nMSAD for Ni: {round(msad_ni,4)} pm^2\n')
    
    oszicar = read(osz_dir)[split_a]
    e_ground = ''.join(oszicar).split()[split_b]
    print(e_ground)
    
    return param

def energy_ana(pos_dir, osz_dir, tline=8):
    li_sum_pos, strip_pos, pos_coor = get_coor(pos_dir,tline)
    oszicar = read(osz_dir)[split_a]
    e_0 = ''.join(oszicar).split()[-4]
    f = ''.join(oszicar).split()[-6]
    print(f'E0: {e_0}')
    print(f'F: {f}')
    print(f'total num: {li_sum_pos}')
    return e_0, f

pattern_compare = '[ /=/ ]'
def split_str(inputlist):
    para = re.split(pattern_compare, inputlist)[0]
    val = re.split(pattern_compare, inputlist)[3]
    return para, val

def differ_find(file1, file2):
    file1 = read(file1)
    file2 = read(file2)

    dup = [[i, j] for i, j in zip(file1, file2) if split_str(i)[0] == split_str(j)[0]
          and split_str(i)[1] != split_str(j)[1]]
    
    split1 = [split_str(file1[i])[0] for i in range(len(file1))]
    split2 = [split_str(file2[i])[0] for i in range(len(file2))]
    differ1 = [file1[split1.index(i)] for i in split1 if i not in split2]
    differ2 = [file2[split2.index(i)] for i in split2 if i not in split1]
    
    print(f'Value changing: {dup}\nUnique param in file1:{differ1}\nUnique param in file2: {differ2}')
    return dup, differ1, differ2

In [31]:
cont_dir = '/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try1/CONTCAR'
pos_dir = '/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try1/POSCAR'
incar_dir = '/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try1/INCAR'
oszicar_dir = '/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try1/OSZICAR'
oszicar = read(oszicar_dir)[split_a]
e_ground = ''.join(oszicar).split()[split_b]

incar = read(incar_dir)
param_try = msad_ana(cont_dir, pos_dir, oszicar_dir, 3.5447, 3)
#param_try['E_ground'] = e_ground
comprehen_incar = incar.append(param_try)

IndexError: list index out of range

In [33]:
#precise structure

for i in range():      
    cont_dir = f'/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try{i}/CONTCAR'
    pos_dir = f'/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try{i}/POSCAR'
    incar_dir = f'/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try{i}/INCAR'
    oszicar_dir = f'/home/wang/MSAD/OUTPUTS/Co/Co_3_3.5447_3_try{i}/OSZICAR'
    oszicar = read(oszicar_dir)[split_a]
    e_ground= ''.join(oszicar).split()[split_b]

    incar = read(incar_dir)
    param_try = msad_ana_bi(cont_dir, pos_dir, oszicar_dir, 3.5447, 3)
    #param_try['E_ground'] = e_ground
    comprehen_incar = incar.append(param_try)

IndexError: list index out of range

In [56]:
#rough structure

for i in range(100, 114):      
    cont_dir = f'/home/wang/MSAD/OUTPUTS/Cr45Co20Ni35/Cr45Co20Ni35_3_3.535_3_try{i}/CONTCAR'
    pos_dir = f'/home/wang/MSAD/OUTPUTS/Cr45Co20Ni35/Cr45Co20Ni35_3_3.535_3_try{i}/POSCAR'
    incar_dir = f'/home/wang/MSAD/OUTPUTS/Cr45Co20Ni35/Cr45Co20Ni35_3_3.535_3_try{i}/INCAR'
    oszicar_dir = f'/home/wang/MSAD/OUTPUTS/Cr45Co20Ni35/Cr45Co20Ni35_3_3.535_3_try{i}/OSZICAR'
    oszicar = read(oszicar_dir)[split_a]
    e_ground = ''.join(oszicar).split()[split_b]

    incar = read(incar_dir)
    param_try = msad_ana(cont_dir, pos_dir, oszicar_dir, 3.535, 3)
    #param_try['E_ground'] = e_ground
    comprehen_incar = incar.append(param_try)

Num of atoms in CONTCAR: 108
Num of atoms in POSCAR: 108
Num of Cr, Co, Ni: [49.0, 21.0, 38.0]
MSAD value: 52.5006 pm^2
MSAD for Cr: 87.8552 pm^2
MSAD for Co: 31.5414 pm^2
MSAD for Ni: 18.4947 pm^2

-.82283833E+03
Num of atoms in CONTCAR: 108
Num of atoms in POSCAR: 108
Num of Cr, Co, Ni: [49.0, 21.0, 38.0]
MSAD value: 75.9467 pm^2
MSAD for Cr: 136.6412 pm^2
MSAD for Co: 29.2125 pm^2
MSAD for Ni: 23.5095 pm^2

-.82203812E+03
Num of atoms in CONTCAR: 108
Num of atoms in POSCAR: 108
Num of Cr, Co, Ni: [49.0, 21.0, 38.0]
MSAD value: 63.132 pm^2
MSAD for Cr: 107.8657 pm^2
MSAD for Co: 28.3383 pm^2
MSAD for Ni: 24.6771 pm^2

-.82207759E+03
Num of atoms in CONTCAR: 108
Num of atoms in POSCAR: 108
Num of Cr, Co, Ni: [48.0, 22.0, 38.0]
MSAD value: 44.6677 pm^2
MSAD for Cr: 76.0207 pm^2
MSAD for Co: 23.1394 pm^2
MSAD for Ni: 17.5277 pm^2

-0.67137E-04


FileNotFoundError: [Errno 2] No such file or directory: '/home/wang/MSAD/OUTPUTS/Cr45Co20Ni35/Cr45Co20Ni35_3_3.535_3_try104/OSZICAR'

In [ ]:
#precise structure Cr20Mn30CoNi

for i in range(1,2):      
    cont_dir = f'/home/wang/MSAD/OUTPUTS/Cr20Mn30CoNi/Cr20Mn30CoNi_3_3.508_4_try{i}/CONTCAR'
    pos_dir = f'/home/wang/MSAD/OUTPUTS/Cr20Mn30CoNi/Cr20Mn30CoNi_3_3.508_4_try{i}/POSCAR'
    incar_dir = f'/home/wang/MSAD/OUTPUTS/Cr20Mn30CoNi/Cr20Mn30CoNi_3_3.508_4_try{i}/INCAR'
    oszicar_dir = f'/home/wang/MSAD/OUTPUTS/Cr20Mn30CoNi/Cr20Mn30CoNi_3_3.508_4_try{i}/OSZICAR'
    oszicar = read(oszicar_dir)[split_a]
    e_ground = ''.join(oszicar).split()[split_b]

    incar = read(incar_dir)
    param_try = msad_ana_qua(cont_dir, pos_dir, oszicar_dir, 3.508, 4)
    #param_try['E_ground'] = e_ground
    comprehen_incar = incar.append(param_try)

In [17]:
#SFE energy
i = 6
osz_dir = f'/home/wang/SFE/outputs/Cr20Co60Ni20/Cr20Co60Ni20_3.486_3_raw_try{i}/OSZICAR'
pos_dir = f'/home/wang/SFE/outputs/Cr20Co60Ni20/Cr20Co60Ni20_3.486_3_raw_try{i}/POSCAR'
osz_dir_stack = f'/home/wang/SFE/outputs/Cr20Co60Ni20/Cr20Co60Ni20_3.486_3_stack_try{i}/OSZICAR'
pos_dir_stack = f'/home/wang/SFE/outputs/Cr20Co60Ni20/Cr20Co60Ni20_3.486_3_stack_try{i}/POSCAR'

e0_raw, f_raw = energy_ana(pos_dir, osz_dir, 9)
e0_stack, f_stack = energy_ana(pos_dir_stack, osz_dir_stack, 9)

Num of atoms in POSCAR: 216
E0: -.14875150E+04
F: -.14875306E+04
total num: 216
Num of atoms in POSCAR: 216
E0: -.14875035E+04
F: -.14875218E+04
total num: 216


In [50]:
#SFE energy
i = 3
osz_dir = f'/home/wang/SFE/outputs/Cr40Co40Ni20/Cr40Co40Ni20_3.512_3_raw_try{i}/OSZICAR'
pos_dir = f'/home/wang/SFE/outputs/Cr40Co40Ni20/Cr40Co40Ni20_3.512_3_raw_try{i}/POSCAR'
osz_dir_stack = f'/home/wang/SFE/outputs/Cr40Co40Ni20/Cr40Co40Ni20_3.512_3_stack_try{i}/OSZICAR'
pos_dir_stack = f'/home/wang/SFE/outputs/Cr40Co40Ni20/Cr40Co40Ni20_3.512_3_stack_try{i}/POSCAR'

e0_raw, f_raw = energy_ana(pos_dir, osz_dir, 9)
e0_stack, f_stack = energy_ana(pos_dir_stack, osz_dir_stack, 9)

Num of atoms in POSCAR: 216
E0: -.15909271E+04
F: -.15909366E+04
total num: 216
Num of atoms in POSCAR: 216
E0: -.15914255E+04
F: -.15914293E+04
total num: 216


In [42]:
#SFE energy
i = 5
osz_dir = f'/home/wang/SFE/outputs/Cr45Co5Ni50/Cr45Co5Ni50_3.541_3_raw_try{i}/OSZICAR'
pos_dir = f'/home/wang/SFE/outputs/Cr45Co5Ni50/Cr45Co5Ni50_3.541_3_raw_try{i}/POSCAR'
osz_dir_stack = f'/home/wang/SFE/outputs/Cr45Co5Ni50/Cr45Co5Ni50_3.541_3_stack_try{i}/OSZICAR'
pos_dir_stack = f'/home/wang/SFE/outputs/Cr45Co5Ni50/Cr45Co5Ni50_3.541_3_stack_try{i}/POSCAR'

e0_raw, f_raw = energy_ana(pos_dir, osz_dir, 9)
e0_stack, f_stack = energy_ana(pos_dir_stack, osz_dir_stack, 9)

Num of atoms in POSCAR: 216
E0: -.15440953E+04
F: -.15440939E+04
total num: 216
Num of atoms in POSCAR: 216
E0: -.15447896E+04
F: -.15447949E+04
total num: 216


In [54]:
#SFE energy
i = 3
osz_dir = f'/home/wang/SFE/outputs/Cr30Co20Ni50/Cr30Co20Ni50_3.523_3_raw_try{i}/OSZICAR'
pos_dir = f'/home/wang/SFE/outputs/Cr30Co20Ni50/Cr30Co20Ni50_3.523_3_raw_try{i}/POSCAR'
osz_dir_stack = f'/home/wang/SFE/outputs/Cr30Co20Ni50/Cr30Co20Ni50_3.523_3_stack_try{i}/OSZICAR'
pos_dir_stack = f'/home/wang/SFE/outputs/Cr30Co20Ni50/Cr30Co20Ni50_3.523_3_stack_try{i}/POSCAR'

e0_raw, f_raw = energy_ana(pos_dir, osz_dir, 9)
e0_stack, f_stack = energy_ana(pos_dir_stack, osz_dir_stack, 9)

Num of atoms in POSCAR: 216
E0: -.14678604E+04
F: -.14678618E+04
total num: 216
Num of atoms in POSCAR: 216
E0: -.14681046E+04
F: -.14681045E+04
total num: 216


In [55]:
lattice_parameter = 3.523
milijoule = (float(e0_stack)-float(e0_raw))/(6.241506363e15)
stack_A = 25*math.sqrt(3)/4*(lattice_parameter**2)*(10e-20)
milijoule/stack_A

-2.9119928946126694

In [56]:
#SFE energy
i = 7


#osz_dir_cr = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Cr0+/OSZICAR'
#osz_dir_co = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Co0+/OSZICAR'
#osz_dir_ni = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Ni0+/OSZICAR'

#pos_dir_cr = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Cr0+/POSCAR'
#pos_dir_co = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Co0+/POSCAR'
#pos_dir_ni = f'/home/wang/SFE/outputs/CrCoNi/CrCoNi_3.516_3_stack_try{i}/Ni0+/POSCAR'

e0_stack, f_stack = energy_ana(pos_dir_stack, osz_dir_stack)
#energy_ana(pos_dir_cr, osz_dir_cr)
#energy_ana(pos_dir_co, osz_dir_co)
#energy_ana(pos_dir_ni, osz_dir_ni)

Num of atoms in POSCAR: 144
E0: -.10357056E+04
F: -.10357054E+04
total num: 144


In [10]:
_,_,_ = differ_find(incar_dir11, incar_dir12)

Value changing: [['SIGMA = 0.1', 'SIGMA = 0.2']]
Unique param in file1:[]
Unique param in file2: []


In [ ]:
save_book = load_workbook(file)
    writer = pd.ExcelWriter(file, engine='pywin32', mode='a')
    writer.book = save_book
    
    split_pattern = '[/\\\_]'
    type_dir = re.split(split_pattern, cont_dir)
    lattice_parameter = int(type_dir[-4])/np.power(10, len(list(type_dir[-4]))) + int(type_dir[-5])
    sheet_name = type_dir[-6]
    
    Type = type_dir[-6] + '_' + type_dir[-2]
    df = pd.read_excel(file, sheet_name = sheet_name)
    df.loc[[int(list(type_dir[-2])[-1])-1],['Type', 'Value(pm2)', 'Ground_state_energy', '(Cr, Co, Ni)', 
                                            'Lattice_parameter', 'Units_of_cell', 
                                            'Cr_msad', 'Co_msad', 'Ni_msad']
    ]=[Type, msad, e_ground, f'({cr_num}, {co_num}, {ni_num})',
       lattice_parameter, type_dir[-3], msad_cr, msad_co, msad_ni]
    df.append(pd.Series(name = int(list(type_dir[-2])[-1])))
    
    df.to_excel(writer, sheet_name = sheet_name)
    writer.save()
    writer.close()
    print(df)